# Eating Magical Cakes with Dynamic Programming

> Finding the optimal saving rate with dynamic programming.

- toc: true
- badges: true
- comments: true
- author: David Ngo
- image: images/cake.jpeg
- categories: [economics, mathematics, python]

![](my_icons/fastai_logo.png)

In layman terms, [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) is a method of mathematical optimization—breaking down a complex problem into simpler parts and storing the results to save on computing power. We use dynamic programming to efficiently calculate the Bellman equation and determine optimal solutions.

## Background

Suppose a nice genie has given us a magic cake. The cake's initial size at time 0 is $y_0z_0$, some known number. Once per day, we are allowed to eat a piece of any size from the cake. Overnight, the cake will regenerate with multiplicative random variable $z_t$ whose logarithm is independently and identically distributed $N(\mu, \sigma)$. This means we potentially have an unlimited source of cake! 

Suppose the genie places the condition that we must choose a constant fraction of the cake to eat each day. As rational economists, we want to fully maximize our happiness by eating the right amount of cake each day. For this experiment, we can assume our utility function to be $E\big[\sum_{t=0}^{\infty}\beta^tu(c_t)\big]$. 

## Exercise

We apply the following:

1. Guess that the optimal policy is to eat a constant fraction of the magic cake in every period. 
2. Assume that $u(c) = \frac{c^{1-r}-1}{1-r}$ for $r \geq 0$. Fix $z_0y_0=69$, $r=2$, $\beta = 0.95$, $\mu=0$, and $\sigma = 0.1$. 
4. Using np.random.seed(1234), generate a fixed sequence of 100 log normally distributed random variables. 
5. Use Python to search for the fraction that maximizes the expected reward for that fixed sequence. 

## Terminology

We use the following terminology:

- The **state** space, denoted $X$ is the size of the cake when we start with $(y_0, z_0)$.
- The **control,** denoted $U$, is the piece of any size from the cake that you choose to eat. So we have $u \in U$ such that $0 < u < 1$.
- The **law of motion** represents the transition into the next price. This is the the multiplicative random variable or shock $z_t$, whose logarithm is independently and identically distributed $N(\mu, \sigma)$.
- The **reward** $r$ is $r: X \times Y \rightarrow \mathbb{R}$, so we have the reward r(x,y) for $x \in X$ if we choose some fraction $u \in U$.
- The **discount factor** is $\beta$, how much we discount to forgo current felicity in favor of future felicity.

## Setting up the Dynamic Program

Let us reiterate what we know so far:

<ol>
    <li> The state space is the random size of the cake</li>  
    <li> The control space is how much one eats in period $t$</li> 
    <li> The law of motion is $x_{t+1} = z_{t+1}(x_t-c_t)$, where $x_t$ is the known size of the cake at time $t$. $z_{t+1}$ is lognormally distributed with parameters $\mu$ and $\sigma$.</li> 
    <li> The reward function is $u(c)$.</li>
    <li> The discount factor is $\beta$. </li>   
</ol>

For this problem, we can apply the Bellman equation:

$$
V(x_t) = max_{c_t \in [0,x_t]}\left\{ u(c_t) + \beta E[V\left(z_{t+1}(x_t-c_t)\right)|z_t] \right\}
$$

$V(\cdot)$ represents a value fuction where, for each $c$ given a value $0 \leq u \leq 1$, we maximize the expected value of the optimal plan. Essentially, the bellman equation lets us find the optimal solution of a complex, iterative problems by breaking it down into simpler parts—which is why we use dynmaic programming to solve this problem. An explanation of the bellman equation can be found [here](https://towardsdatascience.com/the-bellman-equation-59258a0d3fa7). 


## Code

Let us first initialize the values of the state space: we assume that $u(c) = \frac{c^{1-r}-1}{1-r}$ for $r \geq 0$. Fix $z_0y_0=69$, $r=2$, $\beta = 0.95$, $\mu=0$, and $\sigma = 0.1$. We then generate a fixed sequence of 100 log normally distributed random variables. 

In [1]:
import numpy as np

# Finding the fraction that maximizes the expected reward for the fixed sequence

z0y0 = 69
r = 2
beta = 0.97
mu = 0
sigma = .1
T = 100

# Generate a fixed sequence of 100 log normally distributed random variables.
np.random.seed(1234)
sequence = np.random.lognormal(mu, sigma, T)

Next, we create the felicity (utility) function. We use this to calculate our happiness for given amount of cake we consume.

In [2]:
# Felicity function

def felicity(c,r):
    """""
    
    Computing the felicity given consumption c
    and reward r
    
    """""
    
    # u(c)
    u = (c**(1-r)-1) / (1-r)
    
    return u

We can then create a function to calculate our utility overtime. 

In [3]:
# Value function

def bigU(c, beta, r, T):
    """""
    
    Computing the sum of utilities
    C is the sequence of consumption
    beta is the discount factor
    r is the reward curvature
    T is the time period
    
    """""
    
    U = 0
    for t in range(T):
        utility = beta**(t)*felicity(c[t], r)
        U+= utility
    
    return U

Now we create a function to calculate the law of motion. We need this to figure out how much the cake will grow in the next period.

In [4]:
# Law of motion

def motion(initial_state, z, share):
    """""
    Computing the sequence of consumption
    z equals a sequence of numbers, representing 
    the random shocks to the size of the cake 
    given the initial state
   
    """""

    c = []
    y_0 = initial_state
    c_0 = share * y_0
    c.append(c_0)
    for t in range(len(z)):
        y_1 = z[t] * (y_0 - c_0)
        c_1 = share*y_1
        c.append(c_1)
        y_0 = y_1
        c_0 = c_1
        
    return c

Finally, we can use the above functions to create a function that determines the fraction of the cake that yields the maximum utility overtime.

In [5]:
# Maximize expected reward function

# Initial values
z0y0 = 69
r = .2
beta = 0.95
mu = 0
sigma = .1
T = 100


def max_cake(initial_state, 
             beta, 
             r, 
             mu,
             sigma,
             T, 
             a, 
             b, 
             increment):

    """""
    
    Finding the number in range [a,b] that provides 
    the maximum utility
    initial_state is the size of the cake at state 0
    beta is the discount factor
    r is some given value to compute utility for this case
    T is the period of time
    mu is the given mean
    sigma is the given standard deviation
    initial_state is the size of the cake at state 0
    Given the range a to b, with increments from a to b
    
    """""
    
    # Generate a fixed sequence of T log normally distributed random variables.
    np.random.seed(1234)
    sequence = np.random.lognormal(mu, sigma, T)
    
    # Array of fractions
    control = np.arange(a, b, increment)
    
    # Empty array
    control_utilities = []
    
    # Compute value function for each constant fraction
    for i in range(len(control)):
        c = motion(initial_state, sequence, control[i])
        utility = bigU(c, beta, r, T)
        control_utilities.append(utility)
        
    # Find the fraction that maximizes utility
    print('The best constant fraction is', 
          control[control_utilities.index(max(control_utilities))], 
          'and the maximum utility is', round(max(control_utilities),2))

max_cake(z0y0, beta, r, mu, sigma, T, .001, 1, .001)

The best constant fraction is 0.22 and the maximum utility is 25.71



## Conclusion

Provided that the optimal policy is to eat a constant fraction of the magic cake in every period, we find that eating 22% of the cake each day maximizes our expected reward for the fixed sequence. Thank you genie!

Things to consider: We made assumptions on our reward function, discount factor, etc. If we were to change the value of these conditions, there is no doubnt that our best constant fraction and maximum utility would also change. And just because we are "happiest" does not mean we are happy at all! I imagine that once I choose a fraction of the cake to eat for each day, I would be compelled to eat the same flavored cake every day for the rest of my life. What a sneaky genie!

## Credits

Photo by [Jacob Schwartz](https://unsplash.com/@rosenberg12?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText) on [Unsplash.](https://unsplash.com/s/photos/cake?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText)
  
This exercise is based on Sargent and Stachurski's article on [Optimal Saving.](https://python.quantecon.org/cake_eating_problem.html) 